In [1]:
using LinearAlgebra
using BenchmarkTools

In [2]:
function kron_mv(x::Array{T, 1}, matrices::Array{T, 2}...) where T<:Number
    m = reduce(kron, matrices)
    return m * x
end

kron_mv (generic function with 1 method)

In [3]:
function kron_mv_low_mem(x::Array{T, 1}, matrices::Array{T, 2}...) where T<:Number
    n = [size(m, 1) for m in matrices]
    l = prod(n)
    r = 1
    
    V = copy(x)
    
    for s in size(n, 1):-1:1
        l ÷= n[s]
        m = matrices[s]
        for k in 1:l, i in 1:r
            slc = (((k-1) * n[s])*r + i):r:(((k * n[s]) - 1)*r + i)
            V[slc] = m * V[slc]
        end
        r *= n[s]
    end
    
    return V
end

kron_mv_low_mem (generic function with 1 method)

In [4]:
num_sites = 6
local_dim = 4

matrices = [rand(ComplexF64, (local_dim, local_dim)) for _ in 1:num_sites]

v = rand(ComplexF64, local_dim ^ num_sites)
v /= sqrt(real(v' * v));

In [5]:
@benchmark kron_mv(v, matrices...)

BenchmarkTools.Trial: 
  memory estimate:  273.13 MiB
  allocs estimate:  15
  --------------
  minimum time:     137.822 ms (14.71% GC)
  median time:      156.726 ms (9.51% GC)
  mean time:        161.509 ms (11.97% GC)
  maximum time:     214.196 ms (37.20% GC)
  --------------
  samples:          31
  evals/sample:     1

In [6]:
@benchmark kron_mv_low_mem(v, matrices...)

BenchmarkTools.Trial: 
  memory estimate:  2.63 MiB
  allocs estimate:  54607
  --------------
  minimum time:     3.183 ms (0.00% GC)
  median time:      3.303 ms (0.00% GC)
  mean time:        3.653 ms (6.38% GC)
  maximum time:     54.949 ms (93.06% GC)
  --------------
  samples:          1365
  evals/sample:     1

In [8]:
isapprox(
    kron_mv(v, matrices...),
    kron_mv_low_mem(v, matrices...)
)

true